In [1]:
import os
import time
import glob
import random
from PIL import Image
import numpy as np
import tensorflow as tf

2023-08-06 03:13:52.573566: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-06 03:13:52.651479: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-06 03:13:52.653201: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-06 03:13:53.739712: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [18]:
def load_image(img_path):
    img = tf.io.read_file(str(img_path))  # Ensure img_path is a string
    img = tf.io.decode_png(img, channels=0, dtype=tf.dtypes.uint16)
    img = tf.image.resize(img, size=(224, 224), antialias=True)
    img = img / 255.0
    return img

In [19]:
def data_path(image_path, hazy_image_path):
    images_path = list(glob.glob(image_path + "*.png"))
    hazy_images_path = list(glob.glob(hazy_image_path + "*.png"))
    
    train_img = []
    val_img = []
    
    # Match each GT image with its corresponding hazy image
    for images in images_path:
        for hazy_images in hazy_images_path:
            train_img.append(hazy_images)
            val_img.append(hazy_images)
            
    total_images = len(train_img)

    temp = list(zip(train_img, val_img))
    np.random.shuffle(temp)
    train_img, val_img = zip(*temp)

    train_img = list(train_img)
    val_img = list(val_img)
    
    # 60:40 : 27:18
    train_img = train_img[:int(total_images * 0.6)]
    train_hazy = val_img[:int(total_images * 0.6)]
    val_img = train_img[int(total_images * 0.6):]
    val_hazy = val_img[int(total_images * 0.6):]

    return train_img, val_img

In [20]:
def dataLoder(train_data, val_data, batch_size):
    train_data_img = tf.data.Dataset.from_tensor_slices([img[1] for img in train_data]).map(lambda data: load_image(data))
    train_data_hazy = tf.data.Dataset.from_tensor_slices([img[0] for img in train_data]).map(lambda data: load_image(data))
    train = tf.data.Dataset.zip((train_data_img, train_data_hazy)).batch(batch_size)
    
    val_data_img = tf.data.Dataset.from_tensor_slices([img[1] for img in val_data]).map(lambda data: load_image(data))
    val_data_hazy = tf.data.Dataset.from_tensor_slices([img[0] for img in val_data]).map(lambda data: load_image(data))
    val = tf.data.Dataset.zip((val_data_img, val_data_hazy)).batch(batch_size)
    
    return train, val

In [21]:
def result(model, hazy_img, img):

    dehazed_img = model(hazy_img, training = True)
    plt.figure(figsize = (15,12))

    display_list = [hazy_img[0], img[0], dehazed_img[0]]
    title = ['Hazy Image', 'Ground Truth', 'Dehazed Image']

    for i in range(3):
        plt.subplot(1, 3, i+1)
        plt.title(title[i])
        plt.imshow(display_list[i])
        plt.axis('off')

    plt.show()

In [41]:
def create_model(input_shape):
    
    model = tf.keras.Sequential()

    # Convolutional Layers
    model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=input_shape))
    model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))

    model.add(tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
    model.add(tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))

    model.add(tf.keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same'))
    model.add(tf.keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same'))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))

    # Upsampling Layers
    model.add(tf.keras.layers.Conv2DTranspose(128, (3, 3), strides=(2, 2), activation='relu', padding='same'))
    model.add(tf.keras.layers.Conv2DTranspose(64, (3, 3), strides=(2, 2), activation='relu', padding='same'))

    # Output Layer
    model.add(tf.keras.layers.Conv2D(3, (1, 1), activation='sigmoid', padding='same'))

    return model

In [42]:
epochs = 15
batch_size = 16
k_init = tf.keras.initializers.random_normal(stddev=0.008, seed = 101)
regularizer = tf.keras.regularizers.L2(1e-4)
b_init = tf.constant_initializer()

train_data, val_data = data_path(image_path="../data/test/target/", hazy_image_path="../data/test/input/")
train, val = dataLoder(train_data, val_data, batch_size)

In [45]:
train_data

['../data/test/input/34-inputs.png',
 '../data/test/input/10-inputs.png',
 '../data/test/input/30-inputs.png',
 '../data/test/input/36-inputs.png',
 '../data/test/input/21-inputs.png',
 '../data/test/input/19-inputs.png',
 '../data/test/input/18-inputs.png',
 '../data/test/input/11-inputs.png',
 '../data/test/input/25-inputs.png',
 '../data/test/input/13-inputs.png',
 '../data/test/input/16-inputs.png',
 '../data/test/input/15-inputs.png',
 '../data/test/input/43-inputs.png',
 '../data/test/input/43-inputs.png',
 '../data/test/input/2-inputs.png',
 '../data/test/input/37-inputs.png',
 '../data/test/input/40-inputs.png',
 '../data/test/input/43-inputs.png',
 '../data/test/input/32-inputs.png',
 '../data/test/input/26-inputs.png',
 '../data/test/input/45-inputs.png',
 '../data/test/input/43-inputs.png',
 '../data/test/input/38-inputs.png',
 '../data/test/input/38-inputs.png',
 '../data/test/input/20-inputs.png',
 '../data/test/input/20-inputs.png',
 '../data/test/input/3-inputs.png',
 '.

In [44]:
input_shape = (224, 224, 3)
optimizer = tf.keras.optimizers.Adam(learning_rate = 1e-4)
net = create_model(input_shape)

train_loss_tracker = tf.keras.metrics.MeanSquaredError(name = "train loss")
val_loss_tracker = tf.keras.metrics.MeanSquaredError(name = "val loss")

In [46]:
def train_model(epochs, train, val, net, train_loss_tracker, val_loss_tracker, optimizer):
    for epoch in range(epochs):
        print("\nStart of epoch %d" % (epoch,), end=' ')
        start_time_epoch = time.time()
        start_time_step = time.time()
        #print(train_batch_haze)

        for train_batch_haze, train_batch_orig in train:
            print(train_batch_haze)
            try:
                with tf.GradientTape() as tape:
                    train_logits = net(train_batch_haze, training=True)
                    loss = mean_squared_error(train_batch_orig, train_logits)

                grads = tape.gradient(loss, net.trainable_weights)
                optimizer.apply_gradients(zip(grads, net.trainable_weights))

                train_loss_tracker.update_state(train_batch_orig, train_logits)
                if step == 0:
                    print('[', end='')
                if step % 64 == 0:
                    print('=', end='')

            except tf.errors.OpError as e:
                print(f"\nError occurred during training step: {e}")

        print(']', end='')
        print('  -  ', end='')
        print('Training Loss: %.4f' % (train_loss_tracker.result()), end='')

        # Validation loop

        for step, (val_batch_haze, val_batch_orig) in enumerate(val):
            try:
                val_logits = net(val_batch_haze, training=False)
                val_loss_tracker.update_state(val_batch_orig, val_logits)

                if step % 32 == 0:
                    display_img(net, val_batch_haze, val_batch_orig)

            except tf.errors.OpError as e:
                print(f"\nError occurred during validation step: {e}")

        print('  -  ', end='')
        print('Validation Loss: %.4f' % (val_loss_tracker.result()), end='')
        print('  -  ', end=' ')
        print("Time taken: %.2fs" % (time.time() - start_time_epoch))

        net.save('trained_model')  # save the model (variables, weights, etc)
        train_loss_tracker.reset_states()
        val_loss_tracker.reset_states()

%%time
train_model(epochs, train, val, net, train_loss_tracker, val_loss_tracker, optimizer)